## 전처리 순서
1. 데이터 피처 확인
2. 결측치 처리
3. 라벨링으로 문자열 데이터 처리
4. 이상치 데이터 처리
5. 숫자열 데이터 시각화
6. StandardScaler와 로그변환 활용하여 데이터 표준정규화
7. EDA 분석(히트맵, 파이차트)
8. 남은 문자열 데이터 원핫 인코딩

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline
from matplotlib import font_manager, rc

path = "c:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname=path).get_name()
rc('font', family=font_name)

# Target 값 : price

In [2]:
car_df_org = pd.read_csv('data/완성.csv')
car_df = car_df_org.copy()
car_df.head()

ParserError: Error tokenizing data. C error: Expected 1 fields in line 4, saw 2


### 1. 데이터 피처 확인

- brand : 제조사
- name : 차종
- type : 연료형태
- km : 달린 km 수
- year : 제조년도
- accident : 사고 여부
- color : 색상 (검정색, 회색, 흰색, 기타)
- location : 판매 장소
- price : 판매 가격
- wd : 차 구동방식
- trim : 차 트림
    - 1 = 기본
    - 2 = 조금 좋은 옵션
    - 3 = 많이 좋은 옵션
    - 4 = 가장 좋은 옵션

In [ ]:
car_df.info()

### 2. 결측치 처리

In [ ]:
print('데이터 세트의 Shape:', car_df.shape)
print('\n전체 feature 들의 type \n',car_df.dtypes.value_counts())

# Null 컬럼 확인 : Null이 있는 컬럼과 그 건수를 내림차순으로 출력
isnull_series = car_df.isnull().sum()
print('\nNull 컬럼과 그 건수:\n ', isnull_series[isnull_series > 0].sort_values(ascending=False))

In [ ]:
#불필요한 컬럼 삭제
car_df.drop(['location'],axis=1,inplace=True)

In [ ]:
car_df['type'] = car_df.apply(lambda x: x['type'].replace('가솔린+LPG','바이퓨얼'), axis = 1)

#### 문자열 데이터 처리 방식
- 라벨링 : accident, wd
- 원핫인코딩 : brand, name, type, color

### 3. 라벨링으로 문자열 데이터 처리

In [ ]:
car_df['accident_e']=0
for i in range(0,len(car_df)):
        if car_df['accident'][i]=='사고':
            car_df['accident_e'][i]=0
        elif car_df['accident'][i]=='단순사고(접촉)':
            car_df['accident_e'][i]=1
        elif car_df['accident'][i]=='단순교환':
            car_df['accident_e'][i]=2
        else: car_df['accident_e'][i]=3
            
car_df['wd_e']=0
for i in range(0,len(car_df)):
        if car_df['wd'][i]=='2WD':
            car_df['wd_e'][i]=0
        else: car_df['wd_e'][i]=1
            
car_df['accident'] = car_df['accident_e']
car_df['wd'] = car_df['wd_e']
car_df['year'] = 2022 - car_df['year']
del car_df['accident_e']
del car_df['wd_e']

### 4. 이상치 데이터 처리

In [ ]:
car_df.drop(car_df.loc[car_df['price']<=100].index, inplace=True)
car_df.drop(car_df.loc[car_df['km']>=400000].index, inplace=True)
car_df.drop(car_df.loc[car_df['price']>=50000].index, inplace=True)

##### 왜곡된 정도 추출 
- 모든 변수를 그래프로 확인 할 수 없으므로 
    - skew() 함수 사용
    - 주의. 숫자형 피처에서 원-핫 인코딩된 카테고리 숫자형 피처 제외


In [ ]:
from scipy.stats import skew

# 숫자형 피처에서 원-핫 인코딩된 카테고리 숫자형 피처 제외
feautres_index = car_df.dtypes[car_df.dtypes != 'object'].index

feautres_index

# 왜도 확인
skew_features = car_df[feautres_index].apply(lambda x : skew(x))
print(skew_features.sort_values(ascending=False))

### 5. 숫자열 데이터 시각화

#### km, year, price 변환 전 시각화

In [ ]:
plt.title('km Histogram')
sns.distplot(car_df['km'])
#plt.savefig('km Histogram.png')

In [ ]:
plt.title('year Histogram')
sns.distplot(car_df['year'])
#plt.savefig('year Histogram.png')

In [ ]:
plt.title('Sale Price Histogram')
sns.distplot(car_df['price'])
#plt.savefig('log price Histogram.png')

- 데이터의 분포가 중심에서 왼쪽으로 치우친 형태
- 정규 분포에서 벗어나 있음

### 6. StandardScaler와 로그변환 활용하여 데이터 표준정규화

#### km와 year를 StandardScaler 이용하여 표준화 후 시각화

In [ ]:
# # StandardScaler 이용 표준화해서 변환

# from sklearn.preprocessing import StandardScaler

# car_km_df = car_df[['km','year']]

# # StandardScaler 객체 생성
# scaler = StandardScaler()

# # fit() : 데이터 변환을 위한 기준 정보 설정
# scaler.fit(car_km_df)

# # fit() : 설정된 정보를 이용해 데이터 변환
# car_scaled = scaler.transform(car_km_df)

# # numpy ndarry를 DataFrame으로 변환
# car_df_scaled = pd.DataFrame(data=car_scaled, columns=['km','year'])

# print('feature 들의 평균 값')
# print(car_df_scaled.mean())
# print('\nfeature 들의 분산 값')
# print(car_df_scaled.var())

# # 모든 컬럼 값의 평균이 0에 가깝고 분산이 1에 가까운 값으로 변환

In [ ]:
# plt.title('km Histogram')
# sns.distplot(car_df['km'])

In [ ]:
# # 변환 데이터 삽입
# car_df[['km','year']] = car_df_scaled[['km','year']]
# car_df.dropna(inplace=True)

**타겟값인 Price를 로그변환 후 시각화**

In [ ]:
# 결과값을 로그 변환하고 다시 분포도 확인
plt.title('Log Transformed price Histogram')
price = np.log1p(car_df['price'])
sns.distplot(price)

In [ ]:
car_df['price'] = np.log1p(car_df['price']) #로그변환 값으로 타겟값을 대체

In [ ]:
car_df.head()

### 7. EDA 분석(히트맵, 파이차트)

In [ ]:
plt.figure(figsize=(20,18))

corr = car_df.corr()
sns.heatmap(corr, annot=True, fmt='.1g')

### 파이차트

In [ ]:
#brand_per = pd.Series(brand_per['brand'], index=brand_per.index)
#brand = ['현대','폭스바겐','제네시스','쌍용','르노삼성','랜드로버','미니포드','쉐보레','BMW','벤츠']

brand = car_df_org.groupby('brand')
brand_sr = brand.size()
brand_sr.plot(kind='pie', figsize=(10,10), autopct= '%1.1f%%', textprops={'size':20})
plt.title('brand 분포', size=20)
plt.legend(brand_sr.index, loc='best')
plt.show()
#plt.savefig('brand pi chart.png')

In [ ]:
wd = car_df_org.groupby('wd')
wd_sr = wd.size()
wd_sr.plot(kind='pie', figsize=(10,10), autopct= '%1.1f%%', textprops={'size':20})
plt.title('wd 분포', size=20)
plt.legend(wd_sr.index, loc='best')
plt.show()
#plt.savefig('wd pi chart.png')

In [ ]:
type1 = car_df.groupby('type')
type_sr = type1.size()
type_sr.plot(kind='pie', figsize=(10,10), autopct= '%1.1f%%', textprops={'size':20})
plt.title('type 분포', size=20)
plt.legend(type_sr.index, loc='best')
plt.show()
#plt.savefig('type pi chart.png')

#### 전처리 끝난 후 왜도 재확인

In [ ]:
feautres_index = car_df.dtypes[car_df.dtypes != 'object'].index

feautres_index

# 왜도 확인
skew_features = car_df[feautres_index].apply(lambda x : skew(x))
print(skew_features.sort_values(ascending=False))

### 8. 원핫 인코딩으로 처리

In [ ]:
print('get_dummies() 수행 전 데이터 Shape:', car_df.shape)
car_df_ohe = pd.get_dummies(car_df)
print('get_dummies() 수행 후 데이터 Shape:', car_df_ohe.shape)
car_df_ohe.head()

In [ ]:
car_df.head()

In [ ]:
car_df_ohe.to_csv("./data/최종 전처리.csv", index = False)